In [1]:
#Connect to database
from pymilvus import connections, Role,db, CollectionSchema, FieldSchema, DataType, Collection
_HOST = '161.97.138.217'
_PORT = '19530'

connections.connect(host=_HOST, port=_PORT, db_name="default")
print(db.list_database())

['default']


In [29]:
#Creates the query embeeding
from pymilvus import Collection
collection = Collection("milvus_final")      # Get an existing collection.
collection.load()

In [50]:
#Create query embedding to search
import cohere  
co = cohere.Client("0vrSwragivZlDzyVZhvaHwypUSXBSnEVqcrjoa4m")  
texts = [  
   'Abraham Lincoln?'
]  
response = co.embed(texts=texts, model='embed-multilingual-v2.0')  
embeddings = response.embeddings # All text embeddings 
for emb in embeddings:
    print(emb) # Print embeddings for the first text

[0.25952148, -0.054992676, -0.32128906, 0.2932129, 0.17651367, 0.10571289, 0.20629883, -0.29736328, 0.14550781, 0.080322266, -0.27124023, -0.1730957, -0.30615234, 0.2680664, -0.32373047, -0.18115234, 0.3947754, 0.08337402, -0.43774414, 0.15429688, -0.16882324, 0.33642578, 0.033203125, -0.035095215, -0.13745117, 0.12658691, -0.07611084, 0.22973633, 0.09918213, 0.39624023, 0.2939453, 0.0042915344, 0.011146545, 0.34277344, -0.44482422, 0.24377441, 0.0010499954, 0.11651611, -0.13146973, 0.3791504, 0.11639404, -0.012084961, 0.1907959, 0.07501221, 0.02999878, -0.05609131, 0.27563477, -0.20996094, -0.14050293, -0.033416748, -0.19909668, 0.18017578, -0.22241211, -0.15844727, -0.7036133, 0.31713867, -0.18762207, 0.36816406, 0.096191406, -0.5991211, 0.3449707, -0.29663086, -0.22973633, -0.21105957, 0.22473145, 0.33447266, 0.018173218, 0.22680664, 0.10089111, 0.17614746, 0.019989014, 0.18139648, 0.5395508, -0.10809326, 0.035491943, -0.15124512, 0.38476562, -0.15393066, 0.15075684, 0.0052986145, 0

In [32]:
search_params = {
    "metric_type": "L2", 
    "offset": 5, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}

In [51]:
#take query results
results = collection.search(
    data=embeddings, 
    anns_field="emb", 
    # the sum of `offset` in `param` and `limit` 
    # should be less than 16384.
    param=search_params,
    limit=5,
    expr=None,
    # set the names of the fields you want to 
    # retrieve from the search result.
    output_fields=['title',"text","url"],
    consistency_level="Strong"
)

In [52]:
#print the results
processed=results[0]
for a in range(len(processed)):
    print(processed[a].entity.get("title"))
    print(processed[a].id)
    print(processed[a].distance)
    print(processed[a].entity.get("url"),"\n")
    #print(processed[a].entity.get("text"),"\n")

Jack Abramoff
282241
39.84519958496094
https://simple.wikipedia.org/wiki?curid=321989 

Mary Todd Lincoln
135249
40.003082275390625
https://simple.wikipedia.org/wiki?curid=31218 

E. E. Evans-Pritchard
245631
40.33155059814453
https://simple.wikipedia.org/wiki?curid=725088 

Iran–Contra affair
89046
40.918113708496094
https://simple.wikipedia.org/wiki?curid=432683 

Brett Kavanaugh
91703
41.61083221435547
https://simple.wikipedia.org/wiki?curid=645600 

